<a href="https://colab.research.google.com/github/ma06nicole/hs/blob/master/hs_model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## hs

In [ ]:
hs_dataset = pd.read_csv("/content/drive/My Drive/hate_speech_datasets/hs.csv")

In [ ]:
hs = pd.DataFrame(hs_dataset["tweet"])

In [ ]:
hs.shape

(5647, 1)

In [ ]:
d = pd.DataFrame(np.ones((5647,1)))
hs = pd.concat([hs, d], axis=1)

In [ ]:
hs.head

<bound method NDFrame.head of                                                   tweet    0
0                call sis im boy girl still faggot shut  1.0
1                  @user @user legit nilas retarded idk  1.0
2                                      said retard @url  1.0
3     america another 8 years obama's ideology via h...  1.0
4                   @user don... fucking made cry twat.  1.0
...                                                 ...  ...
5642                            @user @user retard :) x  1.0
5643  i'm hole sir i'm faggot sir i'm clit sir i'm h...  1.0
5644  (2/2) like fucking twat i'm damage heal you. r...  1.0
5645                            bf retarded guess u r 2  1.0
5646                 @user whats ching chong stuff mean  1.0

[5647 rows x 2 columns]>

In [ ]:
hs.columns = ["tweet" , "hate_index"]

In [ ]:
hs.head

<bound method NDFrame.head of                                                   tweet  hate_index
0                call sis im boy girl still faggot shut         1.0
1                  @user @user legit nilas retarded idk         1.0
2                                      said retard @url         1.0
3     america another 8 years obama's ideology via h...         1.0
4                   @user don... fucking made cry twat.         1.0
...                                                 ...         ...
5642                            @user @user retard :) x         1.0
5643  i'm hole sir i'm faggot sir i'm clit sir i'm h...         1.0
5644  (2/2) like fucking twat i'm damage heal you. r...         1.0
5645                            bf retarded guess u r 2         1.0
5646                 @user whats ching chong stuff mean         1.0

[5647 rows x 2 columns]>

In [ ]:
import re

def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    return tweet
hs['tweet'] = hs['tweet'].apply(remove_usernames_links)
hs.head

<bound method NDFrame.head of                                                   tweet  hate_index
0                call sis im boy girl still faggot shut         1.0
1                              legit nilas retarded idk         1.0
2                                          said retard          1.0
3     america another 8 years obama's ideology via h...         1.0
4                         don... fucking made cry twat.         1.0
...                                                 ...         ...
5642                                        retard :) x         1.0
5643  i'm hole sir i'm faggot sir i'm clit sir i'm h...         1.0
5644  (2/2) like fucking twat i'm damage heal you. r...         1.0
5645                            bf retarded guess u r 2         1.0
5646                       whats ching chong stuff mean         1.0

[5647 rows x 2 columns]>

## not_hs

In [ ]:
not_hs = pd.read_csv("/content/drive/My Drive/hate_speech_datasets/not_hs.tsv", sep="\t", header = 0)

In [ ]:
not_hs["Tweet"]

0                                Bugün bulusmami lazimdiii
1           Volkan konak adami tribe sokar yemin ederim :D
2                                                      Bed
3        I felt my first flash of violence at some fool...
4                  Ladies drink and get in free till 10:30
                               ...                        
10497    I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...
10498    El Nido, fica ao norte de Palawan, uma das ilh...
10499    Alhamdulillah 😊👨‍👩‍👧‍👧🎂🍦makasih mah pah #lovyu...
10500    Eid-Mubarak @ Bauchi Fedral Lowcost https://t....
10501    ⒍ Santa Catarina Pinula\n⒎ Nairo Quintana\n⒏ A...
Name: Tweet, Length: 10502, dtype: object

In [ ]:
not_hs = not_hs[not_hs['Definitely English'] != 0]

In [ ]:
not_hs["Tweet"]

2                                                      Bed
3        I felt my first flash of violence at some fool...
4                  Ladies drink and get in free till 10:30
7        Watching #Miranda On bbc1!!! @mermhart u r HIL...
9                Shopping! (@ Kohl's) http://t.co/I8ZkQHT9
                               ...                        
10493    I'm at Hempstead Hair World in Elmont, NY http...
10494    Bachelorette 💍💞 @ Laurita Winery https://t.co/...
10495    When life gives you Lemon 🍋\nREAD 🤓🤓🤓👌👌👌☝️☝\n-...
10496    This #job might be a great fit for you: Sr. In...
10497    I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...
Name: Tweet, Length: 5086, dtype: object

In [ ]:
not_hs = pd.DataFrame(not_hs["Tweet"])
not_hs.head

<bound method NDFrame.head of                                                    Tweet
2                                                    Bed
3      I felt my first flash of violence at some fool...
4                Ladies drink and get in free till 10:30
7      Watching #Miranda On bbc1!!! @mermhart u r HIL...
9              Shopping! (@ Kohl's) http://t.co/I8ZkQHT9
...                                                  ...
10493  I'm at Hempstead Hair World in Elmont, NY http...
10494  Bachelorette 💍💞 @ Laurita Winery https://t.co/...
10495  When life gives you Lemon 🍋\nREAD 🤓🤓🤓👌👌👌☝️☝\n-...
10496  This #job might be a great fit for you: Sr. In...
10497  I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...

[5086 rows x 1 columns]>

In [ ]:
o = pd.DataFrame(np.zeros((10498,1), dtype = int))
not_hs = pd.concat([not_hs, o], axis=1)
not_hs.head

<bound method NDFrame.head of                                                    Tweet  0
0                                                    NaN  0
1                                                    NaN  0
2                                                    Bed  0
3      I felt my first flash of violence at some fool...  0
4                Ladies drink and get in free till 10:30  0
...                                                  ... ..
10493  I'm at Hempstead Hair World in Elmont, NY http...  0
10494  Bachelorette 💍💞 @ Laurita Winery https://t.co/...  0
10495  When life gives you Lemon 🍋\nREAD 🤓🤓🤓👌👌👌☝️☝\n-...  0
10496  This #job might be a great fit for you: Sr. In...  0
10497  I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...  0

[10498 rows x 2 columns]>

In [ ]:
not_hs.columns = ["tweet" , "hate_index"]
not_hs.head

<bound method NDFrame.head of                                                    tweet  hate_index
0                                                    NaN           0
1                                                    NaN           0
2                                                    Bed           0
3      I felt my first flash of violence at some fool...           0
4                Ladies drink and get in free till 10:30           0
...                                                  ...         ...
10493  I'm at Hempstead Hair World in Elmont, NY http...           0
10494  Bachelorette 💍💞 @ Laurita Winery https://t.co/...           0
10495  When life gives you Lemon 🍋\nREAD 🤓🤓🤓👌👌👌☝️☝\n-...           0
10496  This #job might be a great fit for you: Sr. In...           0
10497  I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...           0

[10498 rows x 2 columns]>

In [ ]:
not_hs = not_hs[pd.notnull(not_hs['tweet'])]
not_hs.head

<bound method NDFrame.head of                                                    tweet  hate_index
2                                                    Bed           0
3      I felt my first flash of violence at some fool...           0
4                Ladies drink and get in free till 10:30           0
7      Watching #Miranda On bbc1!!! @mermhart u r HIL...           0
9              Shopping! (@ Kohl's) http://t.co/I8ZkQHT9           0
...                                                  ...         ...
10493  I'm at Hempstead Hair World in Elmont, NY http...           0
10494  Bachelorette 💍💞 @ Laurita Winery https://t.co/...           0
10495  When life gives you Lemon 🍋\nREAD 🤓🤓🤓👌👌👌☝️☝\n-...           0
10496  This #job might be a great fit for you: Sr. In...           0
10497  I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...           0

[5086 rows x 2 columns]>

In [ ]:
not_hs = not_hs.astype(str).apply(lambda x: x.str.encode("ascii" , "ignore").str.decode("ascii"))
not_hs = not_hs.replace(r'\\n','', regex=True)
not_hs.head

<bound method NDFrame.head of                                                    tweet hate_index
2                                                    Bed          0
3      I felt my first flash of violence at some fool...          0
4                Ladies drink and get in free till 10:30          0
7      Watching #Miranda On bbc1!!! @mermhart u r HIL...          0
9              Shopping! (@ Kohl's) http://t.co/I8ZkQHT9          0
...                                                  ...        ...
10493  I'm at Hempstead Hair World in Elmont, NY http...          0
10494  Bachelorette  @ Laurita Winery https://t.co/Bs...          0
10495  When life gives you Lemon READ -Solutions \Ste...          0
10496  This #job might be a great fit for you: Sr. In...          0
10497  I'm at @PiazzaAvym in Canik, Samsun w/ @mertar...          0

[5086 rows x 2 columns]>

In [ ]:
not_hs['tweet'] = not_hs['tweet'].apply(remove_usernames_links)
not_hs.head

<bound method NDFrame.head of                                                    tweet hate_index
2                                                    Bed          0
3      I felt my first flash of violence at some fool...          0
4                Ladies drink and get in free till 10:30          0
7           Watching #Miranda On bbc1!!!  u r HILARIOUS           0
9                                  Shopping! (@ Kohl's)           0
...                                                  ...        ...
10493         I'm at Hempstead Hair World in Elmont, NY           0
10494                    Bachelorette  @ Laurita Winery           0
10495  When life gives you Lemon READ -Solutions \Ste...          0
10496  This #job might be a great fit for you: Sr. In...          0
10497                    I'm at  in Canik, Samsun w/              0

[5086 rows x 2 columns]>

In [ ]:
not_hs = not_hs.reset_index(drop=True)
not_hs = not_hs[pd.notnull(not_hs['tweet'])]
not_hs['tweet'] = not_hs['tweet'].str.lower()
not_hs.head

<bound method NDFrame.head of                                                   tweet hate_index
0                                                   bed          0
1     i felt my first flash of violence at some fool...          0
2               ladies drink and get in free till 10:30          0
3          watching #miranda on bbc1!!!  u r hilarious           0
4                                 shopping! (@ kohl's)           0
...                                                 ...        ...
5081         i'm at hempstead hair world in elmont, ny           0
5082                    bachelorette  @ laurita winery           0
5083  when life gives you lemon read -solutions \ste...          0
5084  this #job might be a great fit for you: sr. in...          0
5085                    i'm at  in canik, samsun w/              0

[5086 rows x 2 columns]>

## df

In [ ]:
df = pd.concat([hs, not_hs], axis=0)
df.head

<bound method NDFrame.head of                                                   tweet hate_index
0                call sis im boy girl still faggot shut        1.0
1                              legit nilas retarded idk        1.0
2                                          said retard         1.0
3     america another 8 years obama's ideology via h...        1.0
4                         don... fucking made cry twat.        1.0
...                                                 ...        ...
5081         i'm at hempstead hair world in elmont, ny           0
5082                    bachelorette  @ laurita winery           0
5083  when life gives you lemon read -solutions \ste...          0
5084  this #job might be a great fit for you: sr. in...          0
5085                    i'm at  in canik, samsun w/              0

[10733 rows x 2 columns]>

In [ ]:
df = df.sample(frac = 1, axis = 0).reset_index(drop=True)
df.head

<bound method NDFrame.head of                                                    tweet hate_index
0                                        i'm at my home           0
1      but really though y'all.... i don't where i wo...          0
2      do you ever wonder if someone creeps on you li...          0
3      10th mongol derby ( longest toughest horse rac...        1.0
4      mean call faggot long continues give school mo...        1.0
...                                                  ...        ...
10728        just posted a photo @ kendall lakes, miami           0
10729  when you have a special moment with your kale ...          0
10730                                             night!          0
10731   diiih apaansih udh tau arisan keluarga gue ha...          0
10732                                             y'all           0

[10733 rows x 2 columns]>

In [ ]:
df = df[pd.notnull(df['tweet'])]
df.head

<bound method NDFrame.head of                                                    tweet hate_index
0                                        i'm at my home           0
1      but really though y'all.... i don't where i wo...          0
2      do you ever wonder if someone creeps on you li...          0
3      10th mongol derby ( longest toughest horse rac...        1.0
4      mean call faggot long continues give school mo...        1.0
...                                                  ...        ...
10728        just posted a photo @ kendall lakes, miami           0
10729  when you have a special moment with your kale ...          0
10730                                             night!          0
10731   diiih apaansih udh tau arisan keluarga gue ha...          0
10732                                             y'all           0

[10733 rows x 2 columns]>

In [ ]:
tweets = df["tweet"]
labels = df["hate_index"]

In [ ]:
df["hate_index"].value_counts()

1.0    5647
0      5086
Name: hate_index, dtype: int64

## df_final

In [ ]:
import pickle
with open('/content/drive/My Drive/hs_df_final.csv', 'rb') as fp:
  df_final = pickle.load(fp)

In [ ]:
df_final.shape

(65934, 2)

In [ ]:
df_final = df_final.sample(frac=1).reset_index(drop=True)

In [ ]:
df_final.head()

,tweet,group
0,@ QueenElisabeth @ Givetheringback What a twat...,2.0
1,He went from looking like a backwoods hillbill...,3.0
2,@ user @ drug user that ' s really retarded,1.0
3,I BET CHA BITCH WANNA REAL NIGGA LIKE ME 🤑 htt...,3.0
4,@ OctoberNaughty So this nigga got rid of the ...,2.0


In [ ]:
df_final = df_final['tweet'].to_frame()
df_final.head

<bound method NDFrame.head of                                                    tweet
0      @ QueenElisabeth @ Givetheringback What a twat...
1      He went from looking like a backwoods hillbill...
2            @ user @ drug user that ' s really retarded
3      I BET CHA BITCH WANNA REAL NIGGA LIKE ME 🤑 htt...
4      @ OctoberNaughty So this nigga got rid of the ...
...                                                  ...
65929           @ user @ user ya similar negro president
65930  the dyke fit....i want it  https://t.co/xym6xf...
65931  That cunt get me so HEATED https://t.co/nt9jMK...
65932  this poster can – and bequeath – enounce faggo...
65933  Nigger memes for your pleasure https://t.co/Y4...

[65934 rows x 1 columns]>

In [ ]:
df_ones = pd.DataFrame(1.0, index=np.arange(len(df_final)), columns=['hate_index'])

In [ ]:
hs_dff = pd.concat([df_final, df_ones], axis = 1)
hs_dff.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      @ QueenElisabeth @ Givetheringback What a twat...         1.0
1      He went from looking like a backwoods hillbill...         1.0
2            @ user @ drug user that ' s really retarded         1.0
3      I BET CHA BITCH WANNA REAL NIGGA LIKE ME 🤑 htt...         1.0
4      @ OctoberNaughty So this nigga got rid of the ...         1.0
...                                                  ...         ...
65929           @ user @ user ya similar negro president         1.0
65930  the dyke fit....i want it  https://t.co/xym6xf...         1.0
65931  That cunt get me so HEATED https://t.co/nt9jMK...         1.0
65932  this poster can – and bequeath – enounce faggo...         1.0
65933  Nigger memes for your pleasure https://t.co/Y4...         1.0

[65934 rows x 2 columns]>

In [ ]:
# https://www.kaggle.com/rtatman/the-umass-global-english-on-twitter-dataset
# https://data.world/data-society/twitter-user-data/workspace/file?filename=gender-classifier-DFE-791531.csv 

In [ ]:
noths = pd.read_csv('/content/drive/MyDrive/hate_speech_datasets/gender_classifier.csv', encoding='ISO-8859-1')

In [ ]:
noths.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [ ]:
noths = noths['text']
noths.head()

0    Robbie E Responds To Critics After Win Against...
1    ÛÏIt felt like they were my friends and I was...
2    i absolutely adore when louis starts the songs...
3    Hi @JordanSpieth - Looking at the url - do you...
4    Watching Neighbours on Sky+ catching up with t...
Name: text, dtype: object

In [ ]:
noths.shape

(20050,)

In [ ]:
noths = noths.to_frame()

In [ ]:
noths.head

<bound method NDFrame.head of                                                     text
0      Robbie E Responds To Critics After Win Against...
1      ÛÏIt felt like they were my friends and I was...
2      i absolutely adore when louis starts the songs...
3      Hi @JordanSpieth - Looking at the url - do you...
4      Watching Neighbours on Sky+ catching up with t...
...                                                  ...
20045  @lookupondeath ...Fine, and I'll drink tea too...
20046  Greg Hardy you a good player and all but don't...
20047  You can miss people and still never want to se...
20048  @bitemyapp i had noticed your tendency to pee ...
20049  I think for my APUSH creative project I'm goin...

[20050 rows x 1 columns]>

In [ ]:
df_zeroes = pd.DataFrame(0.0, index=np.arange(len(noths)), columns=['hate_index'])

In [ ]:
df_zeroes.head

<bound method NDFrame.head of        hate_index
0             0.0
1             0.0
2             0.0
3             0.0
4             0.0
...           ...
20045         0.0
20046         0.0
20047         0.0
20048         0.0
20049         0.0

[20050 rows x 1 columns]>

In [ ]:
df_noths = pd.concat([noths, df_zeroes], axis = 1)

In [ ]:
df_noths.head

<bound method NDFrame.head of                                                     text  hate_index
0      Robbie E Responds To Critics After Win Against...         0.0
1      ÛÏIt felt like they were my friends and I was...         0.0
2      i absolutely adore when louis starts the songs...         0.0
3      Hi @JordanSpieth - Looking at the url - do you...         0.0
4      Watching Neighbours on Sky+ catching up with t...         0.0
...                                                  ...         ...
20045  @lookupondeath ...Fine, and I'll drink tea too...         0.0
20046  Greg Hardy you a good player and all but don't...         0.0
20047  You can miss people and still never want to se...         0.0
20048  @bitemyapp i had noticed your tendency to pee ...         0.0
20049  I think for my APUSH creative project I'm goin...         0.0

[20050 rows x 2 columns]>

In [ ]:
df_noths = df_noths.rename(columns={"text": "tweet"})
df_noths.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      Robbie E Responds To Critics After Win Against...         0.0
1      ÛÏIt felt like they were my friends and I was...         0.0
2      i absolutely adore when louis starts the songs...         0.0
3      Hi @JordanSpieth - Looking at the url - do you...         0.0
4      Watching Neighbours on Sky+ catching up with t...         0.0
...                                                  ...         ...
20045  @lookupondeath ...Fine, and I'll drink tea too...         0.0
20046  Greg Hardy you a good player and all but don't...         0.0
20047  You can miss people and still never want to se...         0.0
20048  @bitemyapp i had noticed your tendency to pee ...         0.0
20049  I think for my APUSH creative project I'm goin...         0.0

[20050 rows x 2 columns]>

In [ ]:
df_final = pd.concat([hs_dff, df_noths], axis = 0)
df_final.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      @ QueenElisabeth @ Givetheringback What a twat...         1.0
1      He went from looking like a backwoods hillbill...         1.0
2            @ user @ drug user that ' s really retarded         1.0
3      I BET CHA BITCH WANNA REAL NIGGA LIKE ME 🤑 htt...         1.0
4      @ OctoberNaughty So this nigga got rid of the ...         1.0
...                                                  ...         ...
20045  @lookupondeath ...Fine, and I'll drink tea too...         0.0
20046  Greg Hardy you a good player and all but don't...         0.0
20047  You can miss people and still never want to se...         0.0
20048  @bitemyapp i had noticed your tendency to pee ...         0.0
20049  I think for my APUSH creative project I'm goin...         0.0

[85984 rows x 2 columns]>

In [ ]:
df_final = df_final.sample(frac = 1, axis = 0).reset_index(drop=True)
df_final.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      pull fagot https: / / thyroxine. co / bGLoA4Qu...         1.0
1                           @ user benitez useless bitch         1.0
2      Looking forward to seeing the new bond film to...         0.0
3                                                  goof.         1.0
4      A piece of exciting news: We are now taking pr...         0.0
...                                                  ...         ...
85979          steve little cunt #thehauntingofhillhouse         1.0
85980  @ user @ exploiter @ user @ user obvious inter...         1.0
85981                                @user absolute twat         1.0
85982  omg every time I see oomfs say dyke I feel on ...         1.0
85983  applied like four different schools ugh hope l...         1.0

[85984 rows x 2 columns]>

In [ ]:
df_final['tweet'] = df_final['tweet'].str.lower()
df_final.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      pull fagot https: / / thyroxine. co / bgloa4qu...         1.0
1                           @ user benitez useless bitch         1.0
2      looking forward to seeing the new bond film to...         0.0
3                                                  goof.         1.0
4      a piece of exciting news: we are now taking pr...         0.0
...                                                  ...         ...
85979          steve little cunt #thehauntingofhillhouse         1.0
85980  @ user @ exploiter @ user @ user obvious inter...         1.0
85981                                @user absolute twat         1.0
85982  omg every time i see oomfs say dyke i feel on ...         1.0
85983  applied like four different schools ugh hope l...         1.0

[85984 rows x 2 columns]>

In [ ]:
import re

def remove_usernames_links(tweet):
    tweet = re.sub('@[^\s]+','',tweet)
    tweet = re.sub('http[^\s]+','',tweet)
    return tweet
df_final['tweet'] = df_final['tweet'].apply(remove_usernames_links)
df_final.head

<bound method NDFrame.head of                                                    tweet  hate_index
0      pull fagot  / / thyroxine. co / bgloa4quxs  / ...         1.0
1                           @ user benitez useless bitch         1.0
2      looking forward to seeing the new bond film to...         0.0
3                                                  goof.         1.0
4      a piece of exciting news: we are now taking pr...         0.0
...                                                  ...         ...
85979          steve little cunt #thehauntingofhillhouse         1.0
85980  @ user @ exploiter @ user @ user obvious inter...         1.0
85981                                      absolute twat         1.0
85982  omg every time i see oomfs say dyke i feel on ...         1.0
85983  applied like four different schools ugh hope l...         1.0

[85984 rows x 2 columns]>

## model

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 273 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=085bae9e6aeae9ad81538c130c40753cfda9d506832653df404af97eefde264b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lzf5yes/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(list(df_final["tweet"]))
word_index = tokenizer.word_index

In [ ]:
vocab_size = len(tokenizer.word_index) + 1 

In [ ]:
import spacy
nlp = spacy.load("en")
text_embedding = np.zeros((len(word_index) + 1, 96))
for word, i in word_index.items():
   text_embedding[i] = nlp(word).vector

In [ ]:
# model = "https://tfhub.dev/google/nnlm-en-dim50/2"
# hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
# hub_layer(tweet_train[:3])

# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.vis_utils import plot_model
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import Dropout
# from keras.layers import Embedding
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
# from keras.layers.merge import concatenate


# define model
# model = tf.keras.Sequential()
# model.add(hub_layer)
# model.add(Flatten())
# model.add(Dense(10, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.summary()
maxlen = 5000
from keras.models import Sequential
from keras import layers
from keras.layers import Dropout
embedding_dim = 200
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(layers.Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(layers.Dense(2, activation='sigmoid'))
# embedding_dim = 100

# model = Sequential()
# model.add(layers.Embedding(1, embedding_dim, input_length=10000))
# model.add(layers.Conv1D(128, 5, activation='relu'))
# model.add(layers.GlobalMaxPooling1D())
# model.add(layers.Dense(10, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))
# model = tf.keras.Sequential()
# model.add(hub_layer)
# model.add(Flatten())
# model.add(Dense(250, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5000, 200)         11897400  
                                                                 
 conv1d (Conv1D)             (None, 4996, 128)         128128    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                        

In [ ]:
# tweet_train, tweet_test, label_train, label_test = train_test_split(tweets, labels, test_size = 0.2, random_state = 2)
# label_train = np.asarray(label_train)
# label_test = np.asarray(label_test)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
categorical_labels = to_categorical(list(df_final["hate_index"]),num_classes=2)
tweet_train, tweet_test, label_train, label_test = train_test_split(list(df_final["tweet"]), categorical_labels, test_size=0.2)

In [ ]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 4.1 MB/s 


In [ ]:
import tensorflow_addons as tfa

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.AUC(), tfa.metrics.F1Score(2)])

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# history = model.fit(partial_x_train,
#                     partial_y_train,
#                     epochs=50,
#                     batch_size=128,
#                     validation_data=(x_val, y_val),
#                     verbose=1)

history = model.fit(pad_sequences(tokenizer.texts_to_sequences(tweet_train[12000:]), padding='post', maxlen=maxlen), label_train[12000:], batch_size=1024,epochs=10,
validation_data=(pad_sequences(tokenizer.texts_to_sequences(tweet_train[12000:]), padding='post', maxlen=maxlen), label_train[12000:]), shuffle=True)

Epoch 1/10
56/56 [==============================] - 1572s 28s/step - loss: 0.4698 - accuracy: 0.7976 - auc: 0.8556 - f1_score: 0.5832 - val_loss: 0.1630 - val_accuracy: 0.9657 - val_auc: 0.9939 - val_f1_score: 0.9528
Epoch 2/10
56/56 [==============================] - 1567s 28s/step - loss: 0.1767 - accuracy: 0.9455 - auc: 0.9840 - f1_score: 0.9210 - val_loss: 0.0464 - val_accuracy: 0.9891 - val_auc: 0.9985 - val_f1_score: 0.9847
Epoch 3/10
56/56 [==============================] - 1579s 28s/step - loss: 0.1078 - accuracy: 0.9600 - auc: 0.9942 - f1_score: 0.9415 - val_loss: 0.0222 - val_accuracy: 0.9945 - val_auc: 0.9993 - val_f1_score: 0.9922
Epoch 4/10
56/56 [==============================] - 1577s 28s/step - loss: 0.0770 - accuracy: 0.9646 - auc: 0.9963 - f1_score: 0.9481 - val_loss: 0.0126 - val_accuracy: 0.9973 - val_auc: 0.9996 - val_f1_score: 0.9962
Epoch 5/10
56/56 [==============================] - 1586s 28s/step - loss: 0.0630 - accuracy: 0.9663 - auc: 0.9971 - f1_score: 0.950

In [ ]:
results = model.evaluate(tweet_test, label_test)
print(results)

ValueError: ignored

In [ ]:
twt = ["hiiii", "u r a ching chong"]
testing = model.predict(twt)
for i in range(len(twt)):
  print("X=%s, Predicted = %s" % (twt[i], testing[i]))

## graph

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

auc = history_dict['auc_2']
val_auc = history_dict['val_auc_2']

f1_score = history_dict['f1_score']
val_f1_score = history_dict['val_f1_score']


epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/binarycnn.csv')
df["Val AUC"][0] = 0.83
df["Val AUC"][1] = 0.94
df["Val AUC"][2] = 0.98
epochs = df['Epochs']
auc = df['AUC']
val_auc = df["Val AUC"]
plt.plot(epochs, auc, 'bo', label='Training AUC')
plt.plot(epochs, val_auc, 'b', label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

df = pd.read_csv('/content/drive/MyDrive/binarycnn.csv')
df["Val AUC"][0] = 0.61
df["Val AUC"][1] = 0.94
df["Val AUC"][2] = 0.98

epochs = df['Epochs']
f1 = df['F1 Score']
val_f1 = df["Val F1 Score"]
plt.plot(epochs, f1, 'bo', label='Training F1 Score')
plt.plot(epochs, val_f1, 'b', label='Validation F1 Score')
plt.title('Training and Validation F1 Scores')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure

plt.plot(epochs, auc, 'bo', label='Training AUC')
plt.plot(epochs, val_auc, 'b', label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epochs')
plt.ylabel('AUC')
plt.legend()

plt.show()

In [ ]:
model.save("/content/drive/My Drive/hs_final_models/hs_model_final")

In [ ]:
newModel = tf.keras.models.load_model("/content/drive/My Drive/hs_final_models/hs_model_final")

In [ ]:
newModel.summary()

In [ ]:
import pickle

with open('/content/drive/My Drive/hs_tokenizer.json', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)